## Preprocessing

In [ ]:
import dask.dataframe as dd
edge_list = dd.read_parquet('core_edge_list.parquet')
edge_list_pd = edge_list.compute()

In [ ]:
import networkx as nx
import numpy as np
from scipy.stats import entropy
import pandas as pd

# === Build ID to integer map for memory-efficient graph ===
all_ids = pd.Index(edge_list_pd['source_id']).union(edge_list_pd['target_id'])
id_map = {id_: i for i, id_ in enumerate(all_ids)}
reverse_id_map = {v: k for k, v in id_map.items()}

# Map IDs to integers
edge_list_pd['source_int'] = edge_list_pd['source_id'].map(id_map)
edge_list_pd['target_int'] = edge_list_pd['target_id'].map(id_map)

# Build full graph
G = nx.DiGraph()
G.add_edges_from(zip(edge_list_pd['source_int'], edge_list_pd['target_int']))

# Identify core node ints
core_ids_str = set(edge_list_pd.loc[edge_list_pd['is_core'], 'source_id']).union(
    edge_list_pd.loc[edge_list_pd['is_core'], 'target_id']
)
core_ids_int = [id_map[i] for i in core_ids_str if i in id_map]


In [ ]:
import pickle



# Save edge_list with integer mappings
edge_list_pd.to_parquet(f"edge_list_int.parquet", index=False)

# Save id_map and reverse_id_map as pickle
with open(f"{path}id_map.pkl", "wb") as f:
    pickle.dump(id_map, f)

with open(f"{path}reverse_id_map.pkl", "wb") as f:
    pickle.dump(reverse_id_map, f)

# Save core_ids_int
with open(f"{path}core_ids_int.pkl", "wb") as f:
    pickle.dump(core_ids_int, f)

# Optionally: save graph (large file!)
with open(f"{path}graph_nx_G.pkl", "wb") as f:
    pickle.dump(G, f)


In [ ]:
import json
import dask.dataframe as dd
peripheral_df = dd.read_parquet('peripheral_metadata.parquet')

# === Extract domain from JSON in peripheral_df ===
def extract_domain(field_str):
    try:
        field_dict = json.loads(field_str)
        return field_dict.get("field", {}).get("display_name", None)
    except (json.JSONDecodeError, TypeError):
        return None

peripheral_df['domain'] = peripheral_df['primary_topic'].map(extract_domain, meta=('domain', 'object'))
peripheral_df_pd = peripheral_df[['id', 'domain']].compute()

## Network Building

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.13 environment at: /usr
Audited 11 packages in 98ms

        ***********************************************************************
        The pip install of RAPIDS is complete.

        Please do not run any further installation from the conda based installation methods, as they may cause issues!

        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files.
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        ***********************************************************************
        


In [ ]:
import cugraph

In [ ]:
import dask.dataframe as dd
core_df = dd.read_parquet('df_main_final.parquet')

In [ ]:
import pickle
import pandas as pd

edge_list_pd = pd.read_parquet("edge_list_int.parquet")

with open("id_map.pkl", "rb") as f:
    id_map = pickle.load(f)

with open("reverse_id_map.pkl", "rb") as f:
    reverse_id_map = pickle.load(f)

with open("core_ids_int.pkl", "rb") as f:
    core_ids_int = pickle.load(f)



In [ ]:
import dask.dataframe as dd
import pandas as pd
peripheral_df = dd.read_parquet('peripheral_metadata.parquet')

# Keep only relevant columns
core_meta = core_df[['id', 'publication_year']]
peripheral_meta = peripheral_df[['id', 'publication_year']]

# Concatenate and convert to pandas for joining
meta_df = dd.concat([core_meta, peripheral_meta]).compute()
meta_df = meta_df.drop_duplicates(subset='id')  # Just in case

# Rename for joining
meta_df = meta_df.rename(columns={'id': 'paper_id', 'publication_year': 'year'})

# Load edge list
edge_list_pd = pd.read_parquet('core_edge_list.parquet')

# Join to get source and target years
edge_list_pd = edge_list_pd.merge(meta_df.rename(columns={'paper_id': 'source_id', 'year': 'source_year'}),
                                  on='source_id', how='left')
edge_list_pd = edge_list_pd.merge(meta_df.rename(columns={'paper_id': 'target_id', 'year': 'target_year'}),
                                  on='target_id', how='left')


In [ ]:
future_citations = edge_list_pd[edge_list_pd['source_year'] < edge_list_pd['target_year']]
print(future_citations.shape[0])


132343


In [ ]:
edge_list_pd.shape

(69670379, 5)

In [ ]:
import pickle
with open("id_map.pkl", "rb") as f:
    id_map = pickle.load(f)

In [ ]:
from tqdm import tqdm
import cudf
import pandas as pd
import numpy as np

# === Step 0: Preprocess ===
initial_count = len(edge_list_pd)

edge_list_pd['source_int'] = edge_list_pd['source_id'].map(id_map)
edge_list_pd['target_int'] = edge_list_pd['target_id'].map(id_map)

edge_list_pd = edge_list_pd.dropna(subset=['source_year', 'source_int', 'target_int'])
edge_list_pd['source_year'] = edge_list_pd['source_year'].astype(int)

final_count = len(edge_list_pd)
dropped_count = initial_count - final_count
print(f"Dropped {dropped_count} edges out of {initial_count} ({dropped_count / initial_count:.2%})")

edge_list_cudf = cudf.DataFrame.from_pandas(edge_list_pd)




Dropped 186971 edges out of 69670379 (0.27%)


In [ ]:
del meta_df

In [ ]:
from tqdm import tqdm
import cudf
import cugraph
import pandas as pd

# === Input: Required variables ===
# Assumes these are preloaded:
# edge_list_cudf: cuDF DataFrame with at least source_int, target_int, source_year[, target_year]
# core_ids_int: list or array of node ids to retain
# reverse_id_map: {int_id: original_id} mapping from internal node ID to human-readable ID

# --- Step 0: Init ---
metrics_by_year = []

# Define the years you want to iterate over
years = edge_list_cudf['source_year'].dropna().unique().to_pandas()
years = years[(years >= 2015) & (years <= 2025)].sort_values()

# Ensure core_ids_gpu is a cuDF Series
core_ids_gpu = cudf.Series(core_ids_int)

# --- Step 1: Build publication year lookup for all nodes ---
# Extract source years
pub_year_df = edge_list_cudf[['source_int', 'source_year']].drop_duplicates()
pub_year_df = pub_year_df.rename(columns={'source_int': 'id_int', 'source_year': 'pub_year'})
node_pub_year_map = pub_year_df.to_pandas().set_index('id_int')['pub_year'].to_dict()

# Ensure all years are integers and remove missing
node_pub_year_map = {k: int(v) for k, v in node_pub_year_map.items() if pd.notnull(v)}

# If target years exist, add them too
if 'target_year' in edge_list_cudf.columns:
    target_pub_df = (
        edge_list_cudf[['target_int', 'target_year']]
        .drop_duplicates()
        .rename(columns={'target_int': 'id_int', 'target_year': 'pub_year'})
    )
    target_pub_map = target_pub_df.to_pandas().set_index('id_int')['pub_year'].to_dict()
    target_pub_map = {k: int(v) for k, v in target_pub_map.items() if pd.notnull(v)}
    node_pub_year_map.update(target_pub_map)

# --- Step 2: Loop over years and build yearly citation networks ---
for year in tqdm(years, desc="GPU Computing"):
    df_year_pd = edge_list_cudf.to_pandas()

    # Get publication years of both source and target nodes
    df_year_pd['source_year_full'] = df_year_pd['source_int'].map(node_pub_year_map)
    df_year_pd['target_year_full'] = df_year_pd['target_int'].map(node_pub_year_map)

    # Drop edges missing source or target pub year
    df_year_pd = df_year_pd.dropna(subset=['source_year_full', 'target_year_full'])

    # Keep only edges where both source and target existed by this year
    df_year_pd = df_year_pd[
        (df_year_pd['source_year_full'] <= year) &
        (df_year_pd['target_year_full'] <= year)
    ]

    if df_year_pd.shape[0] == 0:
        continue

    # Convert back to cuDF
    df_year = cudf.DataFrame.from_pandas(df_year_pd)

    # Build graph for this year
    G_year = cugraph.Graph(directed=True)
    G_year.from_cudf_edgelist(
        df_year,
        source='source_int',
        destination='target_int',
        renumber=False,
        store_transposed=True  # recommended for PageRank
    )

    # Centrality metrics
    in_deg_df = G_year.in_degree().rename(columns={'degree': 'in_degree'})
    out_deg_df = G_year.out_degree().rename(columns={'degree': 'out_degree'})
    pr_df = cugraph.pagerank(G_year)
    btw_df = cugraph.betweenness_centrality(G_year, k=1000, normalized=True)

    # Merge metrics
    metrics_df_year = (
        pr_df.merge(in_deg_df, on='vertex', how='left')
             .merge(out_deg_df, on='vertex', how='left')
             .merge(btw_df, on='vertex', how='left')
             .rename(columns={
                 'vertex': 'id_int',
                 'pagerank': 'pagerank',
                 'betweenness_centrality': 'betweenness'
             })
    )

    # Filter to nodes published by this year
    pub_year_series = pd.Series(node_pub_year_map)
    id_year_map = metrics_df_year['id_int'].to_pandas().map(pub_year_series).fillna(9999).astype(int)
    metrics_df_year = metrics_df_year[id_year_map <= year]

    # Filter to core nodes
    metrics_df_year = metrics_df_year[metrics_df_year['id_int'].isin(core_ids_gpu)]

    # Add metadata
    metrics_df_year['year'] = year
    metrics_df_year['id'] = metrics_df_year['id_int'].to_pandas().map(reverse_id_map)

    # Store result
    metrics_by_year.append(metrics_df_year.to_pandas()[[
        'year', 'id', 'in_degree', 'out_degree', 'pagerank', 'betweenness'
    ]])

# --- Step 3: Final output ---
metrics_df = pd.concat(metrics_by_year, ignore_index=True)


GPU Computing:   0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/cugraph/centrality/betweenness_centrality.py:155: UserWarning: Betweenness centrality expects the 'store_transposed' flag to be set to 'False' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)
GPU Computing:   9%|▉         | 1/11 [06:28<1:04:49, 388.96s/it]/usr/local/lib/python3.11/dist-packages/cugraph/centrality/betweenness_centrality.py:155: UserWarning: Betweenness centrality expects the 'store_transposed' flag to be set to 'False' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)
GPU Computing:  18%|█▊        | 2/11 [13:00<58:34, 390.51s/it]  /usr/local/lib/python3.11/dist-packages/cugraph/centrality/betweenness_centrality.py:155: UserWarning: Betweenness centrality expects the 'store_transposed' flag to be set to 'False' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWar

In [ ]:
metrics_df.to_parquet("metrics_df.parquet", index=False)

In [ ]:
import gc
gc.collect()

30

In [ ]:
del df_year

In [ ]:
import sys

def print_variable_sizes():
    """Prints the size of variables in the current scope."""
    print("Variable sizes:")
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in globals().items()), key= lambda x: x[1], reverse=True):
        print(f"{name}: {size} bytes")

print_variable_sizes()

Variable sizes:
df_year_pd: 17187807550 bytes
edge_list_pd: 15463013097 bytes
metrics_df: 9129453571 bytes
id_year_map: 968090228 bytes
reverse_id_map: 671088728 bytes
node_pub_year_map: 671088728 bytes
id_map: 492131760 bytes
pub_year_series: 475152072 bytes
target_pub_map: 335544400 bytes
core_ids_int: 125019096 bytes
future_citations: 27949035 bytes
_i11: 4295 bytes
tqdm: 1688 bytes
_i5: 1110 bytes
_i9: 715 bytes
_i4: 530 bytes
_i15: 375 bytes
_oh: 224 bytes
Out: 224 bytes
years: 208 bytes
_ih: 184 bytes
In: 184 bytes
metrics_by_year: 184 bytes
_i6: 172 bytes
_i3: 169 bytes
f: 168 bytes
_i8: 168 bytes
_i1: 158 bytes
print_variable_sizes: 152 bytes
_iii: 147 bytes
_i12: 147 bytes
__doc__: 113 bytes
__builtin__: 72 bytes
__builtins__: 72 bytes
cugraph: 72 bytes
dd: 72 bytes
pickle: 72 bytes
pd: 72 bytes
cudf: 72 bytes
np: 72 bytes
gc: 72 bytes
sys: 72 bytes
_ii: 71 bytes
_i13: 71 bytes
_i7: 67 bytes
_dh: 64 bytes
get_ipython: 64 bytes
_i2: 63 bytes
_i: 60 bytes
_i10: 60 bytes
_i14: 60

In [ ]:
del meta_df

NameError: name 'meta_df' is not defined

In [ ]:
from tqdm import tqdm

in_core_yearly = []
in_noncore_yearly = []

years = edge_list_cudf['source_year'].unique().to_pandas()
years = years[(years >= 2015) & (years <= 2025)].sort_values()
edge_list_pd['source_year'] = edge_list_pd['source_year'].astype(int)

for year in tqdm(years):
    df_year = edge_list_pd[edge_list_pd['source_year'] <= year].copy()

    df_year['source_is_core'] = df_year['source_int'].isin(core_ids_int)

    # Group by target node
    in_core_deg = (
        df_year[df_year['source_is_core']]
        .groupby('target_int')
        .size()
        .to_dict()
    )
    in_noncore_deg = (
        df_year[~df_year['source_is_core']]
        .groupby('target_int')
        .size()
        .to_dict()
    )

    targets = set(in_core_deg.keys()).union(in_noncore_deg.keys())
    for tid in targets:
        in_core_yearly.append({
            'year': year,
            'id': reverse_id_map[tid],
            'in_degree_core': in_core_deg.get(tid, 0),
            'in_degree_noncore': in_noncore_deg.get(tid, 0)
        })

core_split_df = pd.DataFrame(in_core_yearly)


100%|██████████| 11/11 [06:06<00:00, 33.31s/it]


In [ ]:
core_split_df.to_parquet("core_split_df.parquet", index=False)

In [ ]:
# List of variables you want to keep
whitelist = ['core_split_df', 'metrics_df_full']  # Remove 'core_df' if you don't need it

# Delete all other variables from the global namespace
for var in list(globals()):
    if var not in whitelist and not var.startswith("_"):  # ignore built-ins
        del globals()[var]

In [ ]:
import gc
gc.collect()

0

In [ ]:
import dask.dataframe as dd
import pandas as pd
metrics_df = dd.read_parquet("metrics_df.parquet")
core_split_df = dd.read_parquet("core_split_df.parquet")

In [ ]:
# Merge both sets of metrics
metrics_df_full = metrics_df.merge(core_split_df, on=['id', 'year'], how='left')

# Fill NaNs from merge
metrics_df_full['in_degree_core'] = metrics_df_full['in_degree_core'].fillna(0).astype(int)
metrics_df_full['in_degree_noncore'] = metrics_df_full['in_degree_noncore'].fillna(0).astype(int)


In [ ]:
metrics_df_full.to_parquet('metrics_df_full.parquet')

In [ ]:
del metrics_df

In [ ]:
import dask.dataframe as dd
metrics_df_full = dd.read_parquet('metrics_df_full.parquet')
print(metrics_df_full.head())

   year                                id  in_degree  out_degree  \
0  2015  https://openalex.org/W1044309860          0           0   
1  2015   https://openalex.org/W102875479          0           0   
2  2015   https://openalex.org/W103038265          1           0   
3  2015   https://openalex.org/W101613453          0           0   
4  2015  https://openalex.org/W1003586245          0           0   

       pagerank  betweenness  in_degree_core  in_degree_noncore  
0  8.254617e-09          0.0               0                  0  
1  8.254617e-09          0.0               0                  0  
2  8.514486e-09          0.0               1                  0  
3  8.254617e-09          0.0               0                  0  
4  8.254617e-09          0.0               0                  0  


In [ ]:
import dask.dataframe as dd
core_df = dd.read_parquet('df_main_final.parquet')
core_df.shape[0].compute()

1884132

In [ ]:
# Find IDs in core_df that are not in result_df
missing_ids = core_df[~core_df['id'].isin(result_df['id'])].compute()
missing_ids

,id,doi,title,publication_year,countries_distinct_count,institutions_distinct_count,cited_by_count,cited_by_percentile_year,keywords,counts_by_year,...,fwci,journal_title,prim_topic,prim_field,prim_domain,prim_score,missing_abstract,is_aging,topic,topic_cleaned
2,https://openalex.org/W2579544162,https://doi.org/10.11569/wcjd.v23.i29.4720,Clinical application of double plasma molecula...,2015,0,0,0,"{'min': 0, 'max': 66}",[],[],...,0.0,World Chinese Journal of Digestology,Hepatitis C virus research,Medicine,Health Sciences,0.8271,True,NaN,<NA>,hepatology
9,https://openalex.org/W2550796016,https://doi.org/10.1016/j.ijsu.2016.08.453,Prostate-specific antigen testing in the commu...,2016,1,1,0,"{'min': 0, 'max': 66}",[],[],...,0.0,International Journal of Surgery,Prostate Cancer Diagnosis and Treatment,Medicine,Health Sciences,0.9958,True,NaN,<NA>,oncology
10,https://openalex.org/W4402625801,https://doi.org/10.1016/j.clnesp.2024.07.682,Massive left chylotorax,2024,0,0,0,"{'min': 0, 'max': 79}",[],[],...,0.0,Clinical Nutrition ESPEN,Lymphatic Disorders and Treatments,Medicine,Health Sciences,0.9889,True,NaN,<NA>,surgery
13,https://openalex.org/W2803364913,https://doi.org/10.5505/ptd.2015.59455,Comparision of pupil size in myopic and hypero...,2015,0,0,0,"{'min': 0, 'max': 66}",[{'id': 'https://openalex.org/keywords/pupil-s...,[],...,0.0,Pamukkale Medical Journal,Ophthalmology and Visual Impairment Studies,Medicine,Health Sciences,0.8343,True,NaN,<NA>,ophthalmology
16,https://openalex.org/W2606034669,https://doi.org/10.1016/j.clml.2017.03.200,RalA and RalB are Potential Therapeutic Target...,2017,2,3,0,"{'min': 0, 'max': 63}",[{'id': 'https://openalex.org/keywords/cell-su...,[],...,0.0,"Clinical Lymphoma, Myeloma and Leukemia",Multiple Myeloma Research and Treatments,Medicine,Health Sciences,0.9999,True,NaN,<NA>,oncology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901503,https://openalex.org/W4280532997,https://doi.org/10.1177/2325967121s00538,Type I Tibial Spine Fractures: Not So Simple t...,2022,1,16,0,"{'min': 0, 'max': 59}",[{'id': 'https://openalex.org/keywords/concomi...,[],...,0.0,Orthopaedic Journal of Sports Medicine,Bone fractures and treatments,Medicine,Health Sciences,0.9992,False,1.0,orthopedic surgery,surgery
1901520,https://openalex.org/W4408789364,https://doi.org/10.1530/ec-24-0612,High recovery rate of adrenal function after s...,2025,1,2,0,"{'min': 0, 'max': 97}",[{'id': 'https://openalex.org/keywords/adrenal...,[],...,nan,Endocrine Connections,Pituitary Gland Disorders and Treatments,Medicine,Health Sciences,0.9996,False,1.0,endocrinology,endocrinology
1901523,https://openalex.org/W4280643827,https://doi.org/10.1016/j.jscai.2022.100093,"A-38 | Long Term Patency Rates of Covered, Bal...",2022,1,1,0,"{'min': 0, 'max': 59}",[{'id': 'https://openalex.org/keywords/aortoil...,[],...,0.0,Journal of the Society for Cardiovascular Angi...,Peripheral Artery Disease Management,Medicine,Health Sciences,0.9996,False,1.0,cardiovascular medicine,cardiovascular medicine
1901533,https://openalex.org/W3159353945,https://doi.org/10.1210/jendso/bvab048.1534,The Circadian Clock Gene Bmal1 Modulates Myome...,2021,1,1,0,"{'min': 0, 'max': 56}","[{'id': 'https://openalex.org/keywords/per2', ...",[],...,0.0,Journal of the Endocrine Society,Circadian rhythm and melatonin,Neuroscience,Life Sciences,0.9940,False,1.0,<NA>,obstetrics and gynecology


In [ ]:
missing_ids.columns

Index(['id', 'doi', 'title', 'publication_year', 'countries_distinct_count',
       'institutions_distinct_count', 'cited_by_count',
       'cited_by_percentile_year', 'keywords', 'counts_by_year',
       'journal_issn', 'fwci', 'journal_title', 'prim_topic', 'prim_field',
       'prim_domain', 'prim_score', 'missing_abstract', 'is_aging', 'topic',
       'topic_cleaned'],
      dtype='object')

In [ ]:
# List of variables you want to keep
whitelist = ['core_df_pd', 'result_df', 'core_df', 'metrics_df_full']  # Remove 'core_df' if you don't need it

# Delete all other variables from the global namespace
for var in list(globals()):
    if var not in whitelist and not var.startswith("_"):  # ignore built-ins
        del globals()[var]

# Optionally force garbage collection to free memory
import gc
gc.collect()

result_df = metrics_df_full.merge(core_df, on='id', how='inner')
result_df.to_parquet('result_df_0614.parquet')


In [ ]:
result_df.shape[0].compute() #17061374

8377030

In [ ]:
core_df.shape[0].compute()

1884132

## Data Cleaning

##### Import data

In [ ]:
import dask.dataframe as dd
import pandas as pd
result_df = dd.read_parquet('result_df_0614.parquet')
result_df = result_df.rename(columns={
    'year_x': 'citation_year',
    'year_y': 'publication_year'
})
result_df.columns

Index(['year', 'id', 'in_degree', 'out_degree', 'pagerank', 'betweenness',
       'in_degree_core', 'in_degree_noncore', 'doi', 'title',
       'publication_year', 'countries_distinct_count',
       'institutions_distinct_count', 'cited_by_count',
       'cited_by_percentile_year', 'keywords', 'counts_by_year',
       'journal_issn', 'fwci', 'journal_title', 'prim_topic', 'prim_field',
       'prim_domain', 'prim_score', 'missing_abstract', 'is_aging', 'topic',
       'topic_cleaned'],
      dtype='object')

In [ ]:
result_df[result_df['id'].isna()].shape[0]

<dask_expr.expr.Scalar: expr=(Filter(frame=RenameFrame(frame=ReadParquetFSSpec(9259784), columns={'year_x': 'citation_year', 'year_y': 'publication_year'}), predicate=IsNa(frame=(RenameFrame(frame=ReadParquetFSSpec(9259784), columns={'year_x': 'citation_year', 'year_y': 'publication_year'}))['id']))).size() // 28, dtype=int64>

In [ ]:
result_df['topic_cleaned'].value_counts().compute()

,count
topic_cleaned,
cardiovascular medicine,510733
immunology,315896
infectious diseases,964663
oncology,1281401
rehabilitation medicine,338144
surgery,1133623
hepatology,233936
ophthalmology,170142
other,38


In [ ]:
columns_to_drop = [
    'cited_by_count',
    'cited_by_percentile_year',
    'keywords',
    'counts_by_year',
    'prim_topic',
    'prim_domain',
    'prim_score',
    'topic'
]

# Drop the specified columns
result_df = result_df.drop(columns=columns_to_drop, axis=1)

result_df.columns

Index(['year', 'id', 'in_degree', 'out_degree', 'pagerank', 'betweenness',
       'in_degree_core', 'in_degree_noncore', 'doi', 'title',
       'publication_year', 'countries_distinct_count',
       'institutions_distinct_count', 'journal_issn', 'fwci', 'journal_title',
       'missing_abstract', 'is_aging', 'topic_cleaned'],
      dtype='object')

In [ ]:
row_count = result_df.shape[0].compute()
col_count = result_df.shape[1]

print(f"Shape of result_df: ({row_count}, {col_count})")

Shape of result_df: (8377030, 19)


In [ ]:
result_df = result_df.rename(columns={
    'year': 'citation_year'
})
result_df.head()

,citation_year,id,in_degree,out_degree,pagerank,betweenness,in_degree_core,in_degree_noncore,doi,title,publication_year,countries_distinct_count,institutions_distinct_count,journal_issn,fwci,journal_title,missing_abstract,is_aging,topic_cleaned
0,2015,https://openalex.org/W1008620800,0,0,8.254617e-09,0.0,0,0,None,Distribution and antibiotic resistance of Kleb...,2015,1,1,1005-376x,0.0,Chinese Journal of Microecology,False,1.0,infectious diseases
1,2015,https://openalex.org/W1124505753,0,3,8.254617e-09,0.0,0,0,https://doi.org/10.1016/j.jsurg.2015.05.011,A 19-Year Record of Training Haitian Residents...,2015,2,5,1878-7452,0.327,Journal of Surgical Education,True,NaN,health services research
2,2015,https://openalex.org/W1157292905,0,75,8.254617e-09,0.0,0,0,https://doi.org/10.1016/j.funbio.2015.07.003,A comparison of the community diversity of fol...,2015,1,1,1878-6146,3.603,Fungal Biology,True,NaN,microbiology
3,2015,https://openalex.org/W1197529439,2,26,2.228750e-08,0.0,1,1,https://doi.org/10.4254/wjh.v7.i17.2053,Second line systemic therapies for hepatocellu...,2015,1,1,1948-5182,1.684,World Journal of Hepatology,False,1.0,hepatology
4,2015,https://openalex.org/W1228657578,0,5,8.254617e-09,0.0,0,0,https://doi.org/10.5114/pm.2015.52156,Massive peritoneal cavity calcification in the...,2015,1,2,1643-8876,0.0,Przeglad Menopauzalny,False,1.0,oncology


In [ ]:
result_df['citation_year'] = result_df['citation_year'].astype('int64')
result_df['publication_year'] = result_df['publication_year'].astype('int64')

In [ ]:
result_df = result_df.compute()
result_df.head()

,citation_year,id,in_degree,out_degree,pagerank,betweenness,in_degree_core,in_degree_noncore,doi,title,publication_year,countries_distinct_count,institutions_distinct_count,journal_issn,fwci,journal_title,missing_abstract,is_aging,topic_cleaned
0,2015,https://openalex.org/W1008620800,0,0,8.254617e-09,0.0,0,0,None,Distribution and antibiotic resistance of Kleb...,2015,1,1,1005-376x,0.0,Chinese Journal of Microecology,False,1.0,infectious diseases
1,2015,https://openalex.org/W1124505753,0,3,8.254617e-09,0.0,0,0,https://doi.org/10.1016/j.jsurg.2015.05.011,A 19-Year Record of Training Haitian Residents...,2015,2,5,1878-7452,0.327,Journal of Surgical Education,True,NaN,health services research
2,2015,https://openalex.org/W1157292905,0,75,8.254617e-09,0.0,0,0,https://doi.org/10.1016/j.funbio.2015.07.003,A comparison of the community diversity of fol...,2015,1,1,1878-6146,3.603,Fungal Biology,True,NaN,microbiology
3,2015,https://openalex.org/W1197529439,2,26,2.228750e-08,0.0,1,1,https://doi.org/10.4254/wjh.v7.i17.2053,Second line systemic therapies for hepatocellu...,2015,1,1,1948-5182,1.684,World Journal of Hepatology,False,1.0,hepatology
4,2015,https://openalex.org/W1228657578,0,5,8.254617e-09,0.0,0,0,https://doi.org/10.5114/pm.2015.52156,Massive peritoneal cavity calcification in the...,2015,1,2,1643-8876,0.0,Przeglad Menopauzalny,False,1.0,oncology


In [ ]:
result_df = result_df[result_df['topic_cleaned'] != 'other'].copy()

In [ ]:
result_df.shape

(8376992, 19)

In [ ]:
result_df[result_df['citation_year'] == 2024].shape

(1502926, 19)

In [ ]:
result_df[result_df['citation_year'] == 2024].to_parquet('result_df_2024.parquet')

##### [checkpoint] merge SJR

In [ ]:

print(f"Proportion of missing values in result_df['fwci']: {result_df['fwci'].isnull().mean():.4f}")

Proportion of missing values in result_df['fwci']: 0.0000


In [ ]:
result_df['fwci'] = pd.to_numeric(result_df['fwci'], errors='coerce')

In [ ]:
import pandas as pd
import numpy as np
import re
import unicodedata
import gc

# --- Helper functions ---
def normalize_title(text):
    text = str(text).strip().lower()
    text = unicodedata.normalize("NFKD", text)
    text = re.sub(r'\s+', ' ', text)
    return text

def clean_issn(text):
    return str(text).replace('-', '').strip()

# --- Preprocess result_df ---
df = result_df.copy()
df['citation_year'] = df['citation_year'].astype(int)
df['issn_clean'] = df['journal_issn'].map(clean_issn)
df['journal_title_clean'] = df['journal_title'].map(normalize_title)

# --- Prepare container for matches ---
matched_rows = []

# --- 1. Exact match by ISSN + year ---
for year in range(2015, 2025):
    try:
        sjr = pd.read_csv(f'/content/scimagojr {year}.csv', sep=';', decimal=',', quotechar='"', encoding='utf-8')
    except Exception as e:
        print(f"Error loading SJR for {year}: {e}")
        continue

    sjr['publication_year'] = year
    sjr['journal_title_clean'] = sjr['Title'].map(normalize_title)
    sjr['Issn'] = sjr['Issn'].astype(str)
    sjr_exp = sjr.assign(Issn=sjr['Issn'].str.split(',')).explode('Issn')
    sjr_exp['issn_clean'] = sjr_exp['Issn'].map(clean_issn)

    df_sub = df[df['citation_year'] == year]

    # 1a. Merge by ISSN
    m1 = df_sub.merge(sjr_exp, how='left', on='issn_clean', suffixes=('', '_sjr1'))
    matched = m1[~m1['SJR'].isna()]
    matched_rows.append(matched)

    # 1b. Try matching by title for unmatched
    unmatched = m1[m1['SJR'].isna()][['journal_title_clean', 'citation_year', 'issn_clean', 'journal_issn', 'journal_title']]
    m2 = unmatched.merge(sjr, how='left', on='journal_title_clean', suffixes=('', '_sjr2'))
    matched2 = m2[~m2['SJR'].isna()]
    matched_rows.append(matched2)

    # Clean up to reduce memory
    del sjr, sjr_exp, df_sub, m1, matched, unmatched, m2, matched2
    gc.collect()

# --- 2. Combine all exact matches ---
all_matched = pd.concat(matched_rows, ignore_index=True)
remaining = df.loc[~df.index.isin(all_matched.index)]

del matched_rows
gc.collect()

# --- 3. Fallback match by closest year using ISSN ---
sjr_all = []
for year in range(2015, 2025):
    try:
        df_y = pd.read_csv(f'/content/scimagojr {year}.csv', sep=';', decimal=',', quotechar='"', encoding='utf-8')
        df_y['publication_year'] = year
        df_y['journal_title_clean'] = df_y['Title'].map(normalize_title)
        df_y['Issn'] = df_y['Issn'].astype(str)
        df_y = df_y.assign(Issn=df_y['Issn'].str.split(',')).explode('Issn')
        df_y['issn_clean'] = df_y['Issn'].map(clean_issn)
        sjr_all.append(df_y[['issn_clean', 'SJR', 'publication_year', 'Title']].copy())
    except Exception as e:
        print(f"Error reading SJR fallback for {year}: {e}")
        continue

sjr_issn_all = pd.concat(sjr_all, ignore_index=True)

# Join and find best match by closest year
candidates = remaining.merge(sjr_issn_all, on='issn_clean', how='left')

# Handle suffix: 'publication_year_y' is from SJR data
if 'publication_year_y' not in candidates.columns:
    raise ValueError("Expected 'publication_year_y' after merge but it is missing.")

# Rename for clarity
candidates = candidates.rename(columns={'publication_year_y': 'sjr_year'})

# Compute absolute year difference
candidates['year_diff'] = (candidates['sjr_year'] - candidates['citation_year']).abs()

# Select best match (closest year) per unmatched row
best_match = (
    candidates
    .sort_values('year_diff')
    .groupby(candidates.index, as_index=False)
    .first()
    .drop(columns=['year_diff'])
)

# --- 4. Final combine ---
merged_df = pd.concat([all_matched, best_match], ignore_index=True)

# --- 5. Drop temporary columns ---
temp_cols = ['journal_title_clean', 'issn_clean', 'Issn', 'publication_year_x']
merged_df = merged_df.drop(columns=[col for col in temp_cols if col in merged_df.columns])

# --- 6. Final cleanup ---
del df, sjr_all, sjr_issn_all, remaining, best_match, candidates
gc.collect()


In [ ]:
merged_df.shape

(6365995, 54)

In [ ]:
merged_df.columns

Index(['citation_year', 'id', 'in_degree', 'out_degree', 'pagerank',
       'betweenness', 'in_degree_core', 'in_degree_noncore', 'doi', 'title',
       'publication_year', 'countries_distinct_count',
       'institutions_distinct_count', 'journal_issn', 'fwci', 'journal_title',
       'missing_abstract', 'is_aging', 'topic_cleaned', 'Rank', 'Sourceid',
       'Title', 'Type', 'SJR', 'SJR Best Quartile', 'H index',
       'Total Docs. (2015)', 'Total Docs. (3years)', 'Total Refs.',
       'Total Citations (3years)', 'Citable Docs. (3years)',
       'Citations / Doc. (2years)', 'Ref. / Doc.', '%Female', 'Overton', 'SDG',
       'Country', 'Region', 'Publisher', 'Coverage', 'Categories', 'Areas',
       'publication_year_sjr1', 'journal_title_clean_sjr1',
       'Total Docs. (2016)', 'Total Docs. (2017)', 'Total Docs. (2018)',
       'Total Docs. (2019)', 'Total Docs. (2020)', 'Total Docs. (2021)',
       'Total Docs. (2022)', 'Total Docs. (2023)', 'Total Docs. (2024)',
       'sjr_year'

In [ ]:
# Step 1: drop duplicated SJR columns from sjr2 and sjr3
drop_cols = [col for col in merged_df.columns if (
    col.endswith('_sjr1') or
    col.endswith('_sjr2') or
    col.endswith('_sjr3') or
    re.match(r'Total Docs\. \(\d{4}\)', col)
)]

drop_cols += ['Sourceid', 'Title', 'Type', 'Issn', 'Country', 'Region', 'Publisher', 'Coverage', 'Categories', 'Areas', '']

drop_cols += [
    # Specific SJR fields you listed
    'Total Docs. (2015)', 'Total Docs. (3years)', 'Total Refs.',
    'Total Citations (3years)', 'Citable Docs. (3years)',
    'Citations / Doc. (2years)', 'Ref. / Doc.', '%Female',
    'Overton', 'SDG', 'publication_year_sjr1', 'journal_title_clean_sjr1',

    # All yearly document counts
    'Total Docs. (2016)', 'Total Docs. (2017)', 'Total Docs. (2018)',
    'Total Docs. (2019)', 'Total Docs. (2020)', 'Total Docs. (2021)',
    'Total Docs. (2022)', 'Total Docs. (2023)', 'Total Docs. (2024)',
]

# Drop them
merged_df.drop(columns=drop_cols, inplace=True, errors='ignore')


In [ ]:
filtered_df = merged_df[
    (merged_df['topic_cleaned'].notna())
]

missing_count = filtered_df['SJR Best Quartile'].isna().sum()
total = len(filtered_df)

print(f"Missing matches in ISSN-based merge: {missing_count} out of {total} rows")
print(f"Percentage missing: {missing_count / total:.2%}")
del filtered_df

Missing matches in ISSN-based merge: 0 out of 5514404 rows
Percentage missing: 0.00%


In [ ]:
merged_df.columns

Index(['citation_year', 'id', 'in_degree', 'out_degree', 'pagerank',
       'betweenness', 'in_degree_core', 'in_degree_noncore', 'doi', 'title',
       'publication_year', 'countries_distinct_count',
       'institutions_distinct_count', 'journal_issn', 'fwci', 'journal_title',
       'missing_abstract', 'is_aging', 'topic_cleaned', 'in_core_prop',
       'in_noncore_prop', 'Rank', 'SJR', 'SJR Best Quartile', 'H index',
       'sjr_year'],
      dtype='object')

In [ ]:
result_df[result_df['citation_year'] == 2024].to_parquet('[20251224]main_data.parquet')